In [ ]:
%load_ext autoreload
%autoreload 2

from typing import Dict
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'font.serif': 'Palatino',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

In [ ]:
base_dir = "/Users/vaclav/prog/thesis/data/benchmark_data/"


def read_results(subdir):
    df = pd.read_csv(os.path.join(base_dir, subdir, "results.csv"), index_col=0)

    df["runtime"] = df["name"].str.split("_", n=1, expand=True)[0]

    sparse_rows_mask = df["name"].str.contains("sparse")
    assert df.loc[sparse_rows_mask, "name"].str.contains("0.90_sparse").all(), "Unimplemented sparsity level"
    assert (df.loc[
                sparse_rows_mask, "runtime"] == "DeepSparse").all(), "All sparse models are expected to be DeepSparse"
    df["sparsity"] = 0.0
    df.loc[sparse_rows_mask, "sparsity"] = 0.90

    df.loc[df["runtime"] == "DeepSparse", "runtime"] = "DeepSparse (0%)"
    df.loc[sparse_rows_mask, "runtime"] = "DeepSparse (90%)"

    df["quantization"] = "off"
    df.loc[df["name"].str.contains("quant_static"), "quantization"] = "static"
    df.loc[df["name"].str.contains("quant_dynamic"), "quantization"] = "dynamic"
    # df.loc["sparsity"] = df["name"].str.split("_", n=1, expand=True)[0]

    # Only the largest model
    df = df.loc[df["model_i"] == df["model_i"].max()]

    df["inference_time_ms"] = 1000 * df["inference_time_s"]
    df = df.sort_values("name", kind="stable")

    return df


In [ ]:
plot_dir = "../figures/"


# def make_barplot(df, filename):
#     fig = plt.figure()
#     sns.barplot(data=df, y="name", x='inference_time_s', ax=fig.gca(), ci=95)
#     plt.xlabel("Inference time (s)")
#     plt.ylabel("Runtime library")
#     fig.savefig(os.path.join(plot_dir, filename), bbox_inches="tight")

def make_table(df):
    df_latex_table = (df
                  .groupby(["runtime", "quantization"])
                  .agg({"inference_time_ms": "mean"}) #lambda x: x.quantile(0.99)
                  .reset_index()
                  .pivot(index="runtime",
                         columns="quantization",
                         values="inference_time_ms")
                  )[["off", "static", "dynamic"]]

    # df_latex_table.columns.name = "Quantization"
    # df_latex_table.index.name = "Runtime"
    df_latex_table.columns.name = None
    df_latex_table.index.name = None

    styler = (
        df_latex_table.style
        .format_index(escape="latex")
        .format(precision=2, na_rep="---")
    )

    return styler.to_latex(hrules=True)

def make_plots(df, name, display_name, pdf_metadata: Dict[str, str] = None):
    if pdf_metadata is None:
        pdf_metadata = {}

    ci = 95
    g = sns.catplot(
        data=df, kind="bar",
        y="runtime", x="inference_time_ms", hue="quantization",
        ci=ci, palette="dark", alpha=.6, height=4, aspect=4 / 3,
    )
    plt.xlabel("Inference time (ms)")
    plt.ylabel("Runtime name")

    plt.title(display_name)

    pdf_metadata["ci"] = ci
    pdf_metadata["iterations"] = df["iteration"].max()

    g.fig.savefig(
        os.path.join(plot_dir, f"{name}.pdf"),
        bbox_inches="tight",
        # Metadata doc: https://matplotlib.org/stable/api/backend_pdf_api.html#matplotlib.backends.backend_pdf.PdfFile
        metadata={
            "Title": " ".join([f"{k}={v}" for k, v in pdf_metadata.items()])
        },
        backend="pgf",
    )

    print(make_table(df))

In [ ]:
dirname = "0711-dense-2"
df_dense = read_results(dirname)
make_plots(df_dense, "dense", "Dense model", {"dirname": dirname})

In [ ]:
dirname = "0630-inverted_bottleneck"
df_ib = read_results(dirname)
make_plots(df_ib, "ib", "Inverted bottleneck model", {"dirname": dirname})

In [ ]:
dirname = "0704-dilated_cnn"
df_dilated_cnn = read_results(dirname)
make_plots(df_dilated_cnn, "dilated_cnn", "Dilated CNN model", {"dirname": dirname})